<a href="https://colab.research.google.com/github/Taycanstar/sentiment-analysis-model/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded_files = files.upload()

Saving analysis_data.csv to analysis_data.csv


In [1]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/analysis_data.csv', encoding='ISO-8859-1')



In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from kerastuner import HyperModel
import kerastuner as kt




# # Load the dataset
# df = pd.read_csv('analysis_data.csv', encoding='ISO-8859-1')

# Extract features and labels
X = df['text'].astype(str)
y = pd.Categorical(df['label'], categories=[0, 2, 4])
y = pd.get_dummies(y)

# Split the data into training, validation, and test sets
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.40, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_remaining, y_remaining, test_size=0.50, random_state=42)

# Create TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
validation_data = tf.data.Dataset.from_tensor_slices((X_validation, y_validation))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Define batch size
batch_size = 512

# Shuffle and batch the training data
train_data = train_data.shuffle(buffer_size=10000).batch(batch_size)

# Batch the validation and test data
validation_data = validation_data.batch(batch_size)
test_data = test_data.batch(batch_size)

# Check the shapes
print("Train label shape:", y_train.shape)
print("Validation label shape:", y_validation.shape)
print("Test label shape:", y_test.shape)




<ipython-input-4-a76e6eb89bed>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


Train label shape: (959999, 3)
Validation label shape: (320000, 3)
Test label shape: (320000, 3)


In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/data.csv', encoding='ISO-8859-1')

In [4]:


from transformers import AutoTokenizer,TFBertModel,BertTokenizer
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
from transformers import BertTokenizer
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd

# Assume df is your DataFrame containing 'text' and 'label'


y = pd.get_dummies(df['category'])

# Check the number of classes (should be 3)
num = y.shape[1]
print(num)

# Split the data into training, validation, and test sets
X_train, X_remaining, y_train, y_remaining = train_test_split(df['clean_text'], y, test_size=0.40, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_remaining, y_remaining, test_size=0.50, random_state=42)


# Inspect the data
print("Some text samples:", X_train.head())
print("Data types:", X_train.dtypes)

# Check for and handle null values
if X_train.isnull().any():
    print("Null values found. Filling with empty string.")
    X_train = X_train.fillna('')
if X_validation.isnull().any():
    X_validation = X_validation.fillna('')
if X_test.isnull().any():
    X_test = X_test.fillna('')


bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert = TFBertModel.from_pretrained(bert_model_name)


batch_size = 32  # or any batch size you prefer
max_len = 70

def encode_texts(tokenizer, texts, max_len):
    return tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=max_len,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        return_tensors='tf'
    )

# Encode the data
train_encodings = encode_texts(tokenizer, X_train, max_len)
validation_encodings = encode_texts(tokenizer, X_validation, max_len)
test_encodings = encode_texts(tokenizer, X_test, max_len)

# Prepare TensorFlow datasets
train_data = tf.data.Dataset.from_tensor_slices(({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']}, y_train))
validation_data = tf.data.Dataset.from_tensor_slices(({'input_ids': validation_encodings['input_ids'], 'attention_mask': validation_encodings['attention_mask']}, y_validation))
test_data = tf.data.Dataset.from_tensor_slices(({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']}, y_test))

# Batch the datasets
train_data = train_data.shuffle(buffer_size=10000).batch(batch_size)
validation_data = validation_data.batch(batch_size)
test_data = test_data.batch(batch_size)

# Now you can use train_data, validation_data, and test_data in your model training



input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert.bert(input_ids,attention_mask = input_mask)[0]
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(3,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website
    epsilon=1e-08,
)

model.compile(loss='categorical_crossentropy',  # Changed for multi-class classification
              optimizer=optimizer,
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
NUM_EPOCHS = 2

history = model.fit(train_data, validation_data=validation_data, epochs=NUM_EPOCHS, callbacks=[early_stopping])

# Evaluate the model
results = model.evaluate(test_data, verbose=2)

# Print results
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

# Save as HDF5
# Save the model in TensorFlow SavedModel format
model.save('my_model', save_format='tf')

!zip -r model.zip my_model/

from google.colab import files
files.download('model.zip')


3
Some text samples: 21133    such rubbish schemes will not announced bjp mo...
30605    doesnt belongs from modi bhakts\nthen will sur...
43778    salute our scientists mission shakthi modi com...
253      after jordan india the worlds most intolerant ...
66168    all tamil medias are anti modi and anti bjpall...
Name: clean_text, dtype: object
Data types: object
Null values found. Filling with empty string.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/2


3056/3056 [==============================] - 1576s 502ms/step - loss: 0.2386 - accuracy: 0.9151 - val_loss: 0.1319 - val_accuracy: 0.9559
Epoch 2/2
3056/3056 [==============================] - 1539s 504ms/step - loss: 0.0894 - accuracy: 0.9722 - val_loss: 0.0955 - val_accuracy: 0.9717
1019/1019 - 155s - loss: 0.0943 - accuracy: 0.9702 - 155s/epoch - 152ms/step
loss: 0.094
accuracy: 0.970
  adding: my_model/ (stored 0%)
  adding: my_model/variables/ (stored 0%)
  adding: my_model/variables/variables.data-00000-of-00001 (deflated 13%)
  adding: my_model/variables/variables.index (deflated 79%)
  adding: my_model/fingerprint.pb (stored 0%)
  adding: my_model/keras_metadata.pb (deflated 96%)
  adding: my_model/saved_model.pb (deflated 92%)
  adding: my_model/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
!zip -r model.zip my_model/

  adding: new_model/ (stored 0%)
  adding: new_model/variables/ (stored 0%)
  adding: new_model/variables/variables.data-00000-of-00001 (deflated 13%)
  adding: new_model/variables/variables.index (deflated 79%)
  adding: new_model/fingerprint.pb (stored 0%)
  adding: new_model/keras_metadata.pb (deflated 96%)
  adding: new_model/saved_model.pb (deflated 92%)
  adding: new_model/assets/ (stored 0%)


In [7]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import tensorflow as tf

model = tf.keras.models.load_model('new_model')

print(model.inputs)

ValueError: ignored

In [6]:
from transformers import BertTokenizer
import numpy as np


# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 70  # the max_length used during training

# Load your model
model = tf.keras.models.load_model('my_model')

# Print the model's expected input structure
for input_tensor in model.inputs:
    print(input_tensor)



new_texts = [
    "I had a great day at school today! I aced my math test and my friends threw a surprise party for me. I feel so happy and grateful.",  # Positive
    "Today's science class was fascinating. We learned about space and I'm excited to do more research on this topic. Feeling really inspired!",  # Positive
    "I'm feeling pretty down today. I had an argument with my best friend and it didn't end well. I just wish things were better.",  # Negative
    "I failed my history quiz, and I'm feeling really upset about it. I studied hard but still couldn't remember the dates and events.",  # Negative
    "School was okay today. Nothing special happened. Just attended classes, did some homework, and now I'm getting ready to sleep.",  # Neutral
    "Another regular day at school. Had some tests, they went fine. Nothing exciting or bad happened. Just a normal day.",  # Neutral
]

def encode_texts(tokenizer, texts, max_len):
    return tokenizer.batch_encode_plus(
        texts,
        max_length=max_len,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        return_tensors='tf'
    )

# Tokenize the new texts
new_texts_encodings = encode_texts(tokenizer, new_texts, max_len)

# Make predictions
# Ensure the structure matches
# Example: If the model expects a dictionary with 'input_ids' and 'attention_mask'
prediction_inputs = {
    'input_ids': new_texts_encodings['input_ids'],
    'attention_mask': new_texts_encodings['attention_mask']
}

# Make predictions
predictions = model.predict(prediction_inputs)


# Process the predictions (e.g., take the argmax to get the predicted class index)
predicted_class_indices = np.argmax(predictions, axis=1)

# Map the predicted class indices to your sentiment labels
# Assuming the model's output layer is ordered as [-1 (Negative), 0 (Neutral), 1 (Positive)]
sentiment_labels = {-1: 'Negative', 0: 'Neutral', 1: 'Positive'}
mapped_labels = [sentiment_labels[i-1] for i in predicted_class_indices]  # Subtract 1 to align with your label encoding

# Print the texts with their predicted sentiment
for text, sentiment in zip(new_texts, mapped_labels):
    print(f"Text: {text}\nPredicted Sentiment: {sentiment}\n")


KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.int32, name='input_ids'), name='input_ids', description="created by layer 'input_ids'")
KerasTensor(type_spec=TensorSpec(shape=(None, 70), dtype=tf.int32, name='attention_mask'), name='attention_mask', description="created by layer 'attention_mask'")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 3s 3s/step
Text: I had a great day at school today! I aced my math test and my friends threw a surprise party for me. I feel so happy and grateful.
Predicted Sentiment: Positive

Text: Today's science class was fascinating. We learned about space and I'm excited to do more research on this topic. Feeling really inspired!
Predicted Sentiment: Positive

Text: I'm feeling pretty down today. I had an argument with my best friend and it didn't end well. I just wish things were better.
Predicted Sentiment: Positive

Text: I failed my history quiz, and I'm feeling really upset about it. I studied hard but still couldn't remember the dates and events.
Predicted Sentiment: Negative

Text: School was okay today. Nothing special happened. Just attended classes, did some homework, and now I'm getting ready to sleep.
Predicted Sentiment: Positive

Text: Another regular day at school. Had some tests, they went fine. Nothing exciting or bad happened. Just a norm

In [5]:
import tensorflow as tf
from kerastuner import HyperModel

class CustomRNNModel(HyperModel):
    def __init__(self, encoder, num_classes):
        self.encoder = encoder
        self.num_classes = num_classes

    def build(self, hp):
        # Hyperparameters
        lstm_units = hp.Int('lstm_units', min_value=32, max_value=384, step=32)
        dense_units = hp.Int('dense_units', min_value=16, max_value=64, step=8)
        dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.6, step=0.1)
        reg_rate = hp.Float('reg_rate', min_value=0.0001, max_value=0.01, step=0.0001)

        model = tf.keras.Sequential([
            self.encoder,
            tf.keras.layers.Embedding(input_dim=len(self.encoder.get_vocabulary()), output_dim=64, mask_zero=True),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True)),
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(lstm_units/2))),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(dense_units, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(reg_rate)),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(self.num_classes, activation='softmax')
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model




In [6]:
# Assuming 'encoder' and 'num_classes' are defined as per your dataset and problem
VOCAB_SIZE = 20000

num_classes = 3

encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_data.map(lambda text, label: text))

custom_model = CustomRNNModel(encoder, num_classes)

# Use with Keras Tuner
# Example: using a RandomSearch tuner
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(
    custom_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='model_tuning',
    project_name='rnn_tuning'
)

tuner.search(train_data, validation_data=validation_data, epochs=10, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])


Trial 10 Complete [00h 06m 23s]
val_accuracy: 0.8199812769889832

Best val_accuracy So Far: 0.8213281035423279
Total elapsed time: 01h 21m 39s


In [7]:
# Get the best set of hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print("Best LSTM Units:", best_hps.get('lstm_units'))
print("Best Dense Units:", best_hps.get('dense_units'))
print("Best Dropout Rate:", best_hps.get('dropout_rate'))
print("Best Regularization Rate:", best_hps.get('reg_rate'))



Best LSTM Units: 256
Best Dense Units: 16
Best Dropout Rate: 0.4
Best Regularization Rate: 0.0021


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import tensorflow_hub as hub


embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

VOCAB_SIZE = 20000  # An example of a larger vocabulary size

encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_data.map(lambda text, label: text))

# Assuming 'num_classes' is the number of unique labels

lstm_units = 384  # Best LSTM units
dense_units = 32  # Best dense layer units
dropout_rate = 0.5  # Best dropout rate

num_classes = 3

NUM_EPOCHS = 20

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),  # Reduced units
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(256/2))),  # Reduced units
    tf.keras.layers.Dropout(0.4),  # Increased dropout rate
    tf.keras.layers.Dense(20, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(15, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),  # Added L2 regularization
    tf.keras.layers.Dropout(0.4),  # Increased dropout rate
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# model = tf.keras.Sequential([
#     hub_layer,
#     tf.keras.layers.Reshape(target_shape=(50, 1)),  # Reshape for LSTM layer
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(int(lstm_units / 2))),
#     tf.keras.layers.Dropout(dropout_rate),
#     tf.keras.layers.Dense(dense_units, activation='relu'),
#     tf.keras.layers.Dropout(dropout_rate),
#     tf.keras.layers.Dense(num_classes, activation='softmax')  # Output layer for multi-class
# ])


learning_rate = 0.01  # Set your desired learning rate

# Create an Adam optimizer with the specified learning rate
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy',  # Changed for multi-class classification
              optimizer=adam_optimizer,
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_data, validation_data=validation_data, epochs=NUM_EPOCHS, callbacks=[early_stopping])

# Evaluate the model
results = model.evaluate(test_data, verbose=2)

# Print results
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

# Save as HDF5
# Save the model in TensorFlow SavedModel format
model.save('my_saved_model', save_format='tf')





Epoch 1/20
1875/1875 [==============================] - 126s 59ms/step - loss: 0.4745 - accuracy: 0.7898 - val_loss: 0.4107 - val_accuracy: 0.8164
Epoch 2/20
1875/1875 [==============================] - 89s 47ms/step - loss: 0.4190 - accuracy: 0.8210 - val_loss: 0.4102 - val_accuracy: 0.8173
Epoch 3/20
1875/1875 [==============================] - 88s 47ms/step - loss: 0.4107 - accuracy: 0.8248 - val_loss: 0.4122 - val_accuracy: 0.8161
Epoch 4/20
1875/1875 [==============================] - 87s 47ms/step - loss: 0.4090 - accuracy: 0.8262 - val_loss: 0.4211 - val_accuracy: 0.8149
Epoch 5/20
1875/1875 [==============================] - 87s 47ms/step - loss: 0.4144 - accuracy: 0.8232 - val_loss: 0.4270 - val_accuracy: 0.8095
625/625 - 10s - loss: 0.4094 - accuracy: 0.8180 - 10s/epoch - 16ms/step
loss: 0.409
accuracy: 0.818


In [6]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [5]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])